In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

1. Efficient Data Loading - load using `pandas` with chunking.
1. Data Exploration - 

# Data Understanding and Processing

## Data loading
- The data_date column is converted to a datetime format for proper merging.

In [13]:
%%time
# improved code
# Define the directory where the CSV files are located
data_dir = "../data/raw/"

# List of dataset files
dataset_files = [f"data_set_{i}.csv" for i in range(1, 12)]
risk_factor_files = ["risk_factors_1.csv", "risk_factors_2.csv"]
security_ref_files = ["security_reference_data_w_ret1d_1.csv", "security_reference_data_w_ret1d_2.csv"]

# Function to load data with chunksize
def load_data_with_chunks(file_path, chunksize=100000):
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        # Force the conversion of 'data_date' from integer to datetime
        chunk['data_date'] = pd.to_datetime(chunk['data_date'], format='%Y%m%d')
        chunks.append(chunk)
    return pd.concat(chunks, ignore_index=True)

# Load datasets into a dictionary for easy access
datasets = {file: load_data_with_chunks(os.path.join(data_dir, file)) for file in dataset_files}
risk_factors = {file: load_data_with_chunks(os.path.join(data_dir, file)) for file in risk_factor_files}
security_reference_data = {file: load_data_with_chunks(os.path.join(data_dir, file)) for file in security_ref_files}

CPU times: user 23.5 s, sys: 7.04 s, total: 30.6 s
Wall time: 41.1 s


## Data Exploration
- Basic statistics
- Number of unique stock value, missing values before cleaning

In [18]:
def explore_dataframe(df, name):
    """
    Prints basic statistics and counts for the given DataFrame.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to explore.
    name (str): The name of the DataFrame (for printing).
    """
    print("#" * 50)
    print(f"Exploring {name}")
    print(df.info())
    print(df.describe())
    print("Missing values in each column:")
    print(df.isna().sum())  # Check for missing values
    
    # Count of unique trading dates
    unique_dates_count = df['data_date'].nunique()
    print(f"Count of unique trading dates (data_date): {unique_dates_count}")
    
    # Count of unique security IDs
    if 'security_id' in df.columns:
        unique_stocks_count = df['security_id'].nunique()
        print(f"Count of unique stocks (security_id): {unique_stocks_count}")
    else:
        print("No security_id column in this DataFrame.")

In [19]:
# Explore Security Reference Data
for key, df in security_reference_data.items():
    explore_dataframe(df, key)

##################################################
Exploring security_reference_data_w_ret1d_1.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4311675 entries, 0 to 4311674
Data columns (total 7 columns):
 #   Column               Dtype         
---  ------               -----         
 0   data_date            datetime64[ns]
 1   security_id          int64         
 2   close_price          float64       
 3   volume               float64       
 4   group_id             int64         
 5   in_trading_universe  object        
 6   ret1d                float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 230.3+ MB
None
                           data_date   security_id   close_price  \
count                        4311675  4.311675e+06  4.311675e+06   
mean   2011-12-16 07:23:10.842770432  1.032505e+08  2.750957e+01   
min              2010-01-04 00:00:00  7.790100e+04  1.000100e+00   
25%              2010-12-06 00:00:00  2.377501e+06  5.860000

In [20]:
# Explore Risk Factors
for key, df in risk_factors.items():
    explore_dataframe(df, key)

##################################################
Exploring risk_factors_1.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4311675 entries, 0 to 4311674
Data columns (total 8 columns):
 #   Column       Dtype         
---  ------       -----         
 0   data_date    datetime64[ns]
 1   security_id  int64         
 2   rf1          float64       
 3   rf2          float64       
 4   rf3          float64       
 5   rf4          float64       
 6   rf5          float64       
 7   rf6          float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 263.2 MB
None
                           data_date   security_id           rf1  \
count                        4311675  4.311675e+06  4.311675e+06   
mean   2011-12-16 07:23:10.842770432  1.032505e+08  9.260416e-02   
min              2010-01-04 00:00:00  7.790100e+04 -2.696000e+00   
25%              2010-12-06 00:00:00  2.377501e+06 -7.210000e-01   
50%              2011-12-05 00:00:00  5.025401e+06  7.900000e-

In [21]:
# Explore Datasets
for key, df in datasets.items():
    explore_dataframe(df, key)

##################################################
Exploring data_set_1.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127266 entries, 0 to 127265
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   data_date    127266 non-null  datetime64[ns]
 1   security_id  127266 non-null  int64         
 2   d1           122478 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 2.9 MB
None
                           data_date   security_id            d1
count                         127266  1.272660e+05  1.224780e+05
mean   2014-01-09 08:56:26.997313280  1.854502e+08  2.464821e+03
min              2010-01-04 00:00:00  1.031000e+03 -1.470070e+07
25%              2011-12-07 00:00:00  2.552401e+06 -5.024750e+00
50%              2014-02-18 00:00:00  5.345401e+06  3.731000e+00
75%              2016-02-05 00:00:00  8.688701e+06  4.000000e+01
max              2017-12-29 00:00:0

## Data Cleaning
- Missing values: Drop rows with missing values in data_date and security_id, as these are essential for any merging operations. By analyze missing data - `risk_factors` don't have any NAs. Drop NAs in critical columns and target variables for both `security_reference_data` and `datasets`.
- Verify that after this step, there is no missing value left.

In [22]:
# Handling Missing Values Before Merging
# Drop NAs in critical columns for security reference data
for df in security_reference_data.values():
    df.dropna(subset=['data_date', 'security_id'], inplace=True)
    df.dropna(subset=['ret1d'], inplace=True)  # Drop rows with NAs in the target variable

# Drop NAs in datasets
for df in datasets.values():
    df.dropna(subset=['data_date', 'security_id'], inplace=True)
    df.dropna(subset=[col for col in df.columns if 'd' in col], inplace=True)  # Drop rows with NAs in target variables

## Data Combination
- Concatenate Security Reference Data
- Concatenate Risk Factors
- Index Setting: Before merging, the code sets the index for both the combined security reference data and risk factors to allow for efficient joins.
- Merging: Each dataset is merged with the combined security reference data and risk factors using a left join, ensuring that all relevant features are included.
- Final Combined DataFrame: The final merged_data DataFrame will contain all relevant features from the various datasets and the combined security and risk factor data.

In [28]:
%%time
# Concatenate security reference data
combined_security_reference_data = pd.concat(security_reference_data.values(), ignore_index=True)

# Concatenate risk factors
combined_risk_factors = pd.concat(risk_factors.values(), ignore_index=True)

# Set index for combined DataFrames for efficient merging
combined_security_reference_data.set_index(['data_date', 'security_id'], inplace=True)
combined_risk_factors.set_index(['data_date', 'security_id'], inplace=True)

CPU times: user 1.58 s, sys: 3.11 s, total: 4.69 s
Wall time: 9.12 s


In [29]:
combined_security_reference_data.head()

close_price     volume  group_id in_trading_universe  \
data_date  security_id                                                         
2010-01-04 77901              19.99    72728.0  20102010                   N   
           78001              23.77   460537.0  20101010                   Y   
           78401              19.64  1847102.0  20104020                   Y   
           82901               2.80   219545.0  20301010                   N   
           83501               7.90   389291.0  20201060                   N   

                           ret1d  
data_date  security_id            
2010-01-04 77901        0.025654  
           78001        0.034378  
           78401        0.028272  
           82901        0.060606  
           83501        0.085165

In [31]:
combined_security_reference_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8791161 entries, (Timestamp('2010-01-04 00:00:00'), np.int64(77901)) to (Timestamp('2017-12-29 00:00:00'), np.int64(1118042001))
Data columns (total 5 columns):
 #   Column               Dtype  
---  ------               -----  
 0   close_price          float64
 1   volume               float64
 2   group_id             int64  
 3   in_trading_universe  object 
 4   ret1d                float64
dtypes: float64(3), int64(1), object(1)
memory usage: 369.3+ MB


In [30]:
combined_risk_factors.head()

rf1    rf2    rf3    rf4    rf5    rf6
data_date  security_id                                          
2010-01-04 5311701      0.284 -0.500  1.789  2.962 -0.435 -0.314
           3780701      1.890 -1.330 -0.208  0.775 -0.803 -0.576
           2297301      0.012  0.217  0.285 -0.151  0.883  0.127
           83501        1.185  1.195  2.677 -1.485 -1.236 -1.341
           4159901     -0.335 -2.917  2.548  1.691  0.394 -2.290

In [32]:
combined_risk_factors.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8800015 entries, (Timestamp('2010-01-04 00:00:00'), np.int64(5311701)) to (Timestamp('2017-12-29 00:00:00'), np.int64(1115398701))
Data columns (total 6 columns):
 #   Column  Dtype  
---  ------  -----  
 0   rf1     float64
 1   rf2     float64
 2   rf3     float64
 3   rf4     float64
 4   rf5     float64
 5   rf6     float64
dtypes: float64(6)
memory usage: 436.8 MB


In [33]:
%%time
# Initialize an empty DataFrame for merging datasets
merged_data = pd.DataFrame()

# Merge each dataset with the combined security reference data and risk factors
for file, dataset in datasets.items():
    dataset.set_index(['data_date', 'security_id'], inplace=True)  # Set index for the dataset
    
    # Merge with combined security reference data
    dataset = dataset.join(combined_security_reference_data, how='left')
    
    # Merge with combined risk factors
    dataset = dataset.join(combined_risk_factors, how='left')
    
    # Reset index if needed and store the merged dataset
    dataset.reset_index(inplace=True)
    merged_data = pd.concat([merged_data, dataset], ignore_index=True)

# Now merged_data contains all relevant features from the datasets, security reference data, and risk factors

CPU times: user 2min 44s, sys: 1min 31s, total: 4min 15s
Wall time: 6min 48s


In [34]:
merged_data.head()

,data_date,security_id,d1,close_price,volume,group_id,in_trading_universe,ret1d,rf1,rf2,...,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11
0,2010-01-04,647001,3.4530,13.80,15552.0,25301040.0,N,-0.044983,-1.335,-0.499,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-04,696801,1.5820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-04,1323901,-0.9449,4.00,10119.0,40101015.0,N,0.176506,-1.013,-2.508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-04,1604101,-9.4220,1.42,709.0,40101015.0,N,0.014286,-1.041,-0.760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-01-04,1610801,-3.7690,1.50,219.0,40101015.0,N,0.041667,-0.036,-2.917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32082507 entries, 0 to 32082506
Data columns (total 24 columns):
 #   Column               Dtype         
---  ------               -----         
 0   data_date            datetime64[ns]
 1   security_id          int64         
 2   d1                   float64       
 3   close_price          float64       
 4   volume               float64       
 5   group_id             float64       
 6   in_trading_universe  object        
 7   ret1d                float64       
 8   rf1                  float64       
 9   rf2                  float64       
 10  rf3                  float64       
 11  rf4                  float64       
 12  rf5                  float64       
 13  rf6                  float64       
 14  d2                   float64       
 15  d3                   float64       
 16  d4                   float64       
 17  d5                   float64       
 18  d6                   float64       
 19  d7                 

## Data Saving

In [ ]:

# Example access to a merged dataset
final_dataset = merged_data['data_set_1.csv']  # Example for the first dataset
